In [1]:
import numpy as np
import pandas as pd
import glob
import ipyparallel as p
import os
import ntpath
import ProgressBar as pb
from pymail import alert
import time

In [2]:
exname = 'smex8_sen'
method = '8'
site = 'P301'

In [3]:
files = glob.glob('/Volumes/data/RHESSys_out/P301/%s/*_basin.daily'%exname)

In [4]:
len(files)

150000

In [5]:
params = pd.read_pickle('./data/soil_params_multispinup_%s_smex%s_sen.pcl'%(site,method))

In [6]:
params.combo.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [7]:
params.head()

,MAP,PeakSWE,SMR,TpeakSWE,combo,idx,rd,ref,scenidx,sd
0,0.325,0.521211,0.014403,113.0,1,0,0.011104,2009,0,4.011104
1,0.325,0.521211,0.014403,113.0,2,0,0.111041,2009,1,4.111041
2,0.325,0.521211,0.014403,113.0,3,0,0.222082,2009,2,4.222082
3,0.325,0.521211,0.014403,113.0,4,0,0.444163,2009,3,4.444163
4,0.325,0.521211,0.014403,113.0,5,0,0.888326,2009,4,4.888326


In [8]:
files = pd.DataFrame({'basin_daily':files})

In [9]:
def getIDX(fl):
    return int(fl.split('_')[-2])

In [10]:
files['scenidx'] = files.basin_daily.map(getIDX)

In [11]:
params = pd.merge(params,files, on= 'scenidx')

In [12]:
from rhessys import snowmelt_experiment as smex

In [13]:
params['Tnosnow'] = params.TpeakSWE+np.ceil(params.PeakSWE/params.SMR)

In [14]:
params['date_peakSWE'] = params.apply(smex.DOPEAKSWE2cal,axis=1)
params['date_ONS'] =params.apply(smex.DONS2cal,axis=1)

In [15]:
c = p.Client()
view = c.load_balanced_view()

In [16]:
c.ids

[0, 1, 2, 3, 4, 5, 6, 7]

In [17]:
len(params)

150000

In [18]:
dr = pd.date_range('2008-10-1','2009-10-02',freq='D')

In [19]:
%px dr = pd.date_range('2008-10-1','2009-10-02',freq='D')

In [20]:
def processSM(bd,idx,strt,nd):

    # read in the basin daily data
    data = pd.read_csv(bd,delimiter=' ')

    data.index = pd.DatetimeIndex(dr)

    data['ET'] = data.evap + data.trans
    trans = data.trans.sum()
    evap = data.evap.sum()
    sub = data.snow_subl.sum()
    ET = data.ET.sum() # compute total ET
    PET = data['pet'].sum()
    meanPET = data['pet'].mean()
    Q = data.streamflow.sum() # compute total streamflow 
    Qbf = data.baseflow.sum() # compute total baseflow
    Qrf = data['return'].sum() # compute total returnflow,p
    sm = data.snowmelt.sum()
    rech = data.recharge.sum() # compute total recharge
    sat = data['%sat_area'].mean() # compute the mean saturated area
    p = data.precip.sum() # compute water year precip
    rz_storage = data.rz_storage.mean() # root zone storage
    rz_drainage = data.rz_drainage.mean() # root zone drainage 
    unsat_drainage = data.unsat_drain.mean() # unsaturated zone drainage
    unsat_storage = data.unsat_stor.mean() # unsaturated zone drainage
    sat_def = data.sat_def.mean() # saturation deficite (mm)
    sat_def_z = data.sat_def_z.mean() # saturation deficite depth, is this the depth to ground water?
    gwStore = data['gw.storage'].sum() # gw store
    gwFlux = data['gw.Qout'].mean()
    ETRate = data.ET.mean()
    
    # create full output object
    full = (idx,trans,evap,sub,ET,PET,meanPET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,
            unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate)
    
    data = data.loc[strt:nd] # crop the data to the melt season
    T = data.trans.sum() # compute totoal transpiration
    E = data.evap.sum() # compute total evaporation
    sub = data.snow_subl.sum()
    ET = data.ET.sum() # compute total ET
    PET = data['pet'].sum()
    meanPET = data['pet'].mean()
    Q = data.streamflow.sum() # compute total streamflow 
    Qbf = data.baseflow.sum() # compute total baseflow
    Qrf = data['return'].sum() # compute total returnflow,p
    sm = data.snowmelt.sum()
    rech = data.recharge.sum() # compute total recharge
    sat = data['%sat_area'].mean() # compute the mean saturated area
    p = data.precip.sum() # compute water year precip
    rz_storage = data.rz_storage.mean() # root zone storage
    rz_drainage = data.rz_drainage.mean() # root zone drainage 
    unsat_drainage = data.unsat_drain.mean() # unsaturated zone drainage
    unsat_storage = data.unsat_stor.mean() # unsaturated zone drainage
    sat_def = data.sat_def.mean() # saturation deficite (mm)
    sat_def_z = data.sat_def_z.mean() # saturation deficite depth, is this the depth to ground water?
    gwStore = data['gw.storage'].sum() # gw store
    gwFlux = data['gw.Qout'].mean()
    ETRate = data.ET.mean()
    
    melt = (idx,T,E,sub,ET,PET,meanPET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,
            unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate)
    
    return full,melt

In [21]:
idx = 0
print params.basin_daily[idx]
processSM(params.basin_daily[idx],params.scenidx[idx],params.date_peakSWE[idx],params.date_ONS[idx])

/Volumes/data/RHESSys_out/P301/smex8_sen/smex8_sen_0_basin.daily


((0,
  34.84947,
  81.803955,
  18.039949,
  116.653425,
  1044.7766889999998,
  2.846802967302453,
  616.4584530000001,
  230.15856499999998,
  386.299888,
  558.6806300000001,
  0.0,
  716.4708499999999,
  2.2134157356948063,
  2.732354108991829,
  1941.0120784932024,
  4.322964885558577,
  1946.6444373569475,
  4011.1040799999773,
  388.143045,
  11673.876054,
  0.2587964850136241,
  0.3178567438692099),
 (0,
  3.8754020000000002,
  5.353626000000001,
  5.293211000000001,
  9.229027999999998,
  33.666386,
  0.8859575263157894,
  68.89143899999999,
  31.740791000000005,
  37.15065,
  61.962256999999994,
  0.0,
  0.0,
  2.984500210526316,
  4.052827631578946,
  1942.07875531579,
  6.368756921052633,
  1946.6443399999994,
  4011.10408,
  70.650502,
  1366.295795,
  0.29263349999999994,
  0.2428691578947368))

In [22]:
#res = view.map(processSM,params.basin_daily[0:1000],params.scenidx[0:1000],params.date_peakSWE[0:1000],params.date_ONS[0:1000])
res = view.map(processSM,params.basin_daily,params.scenidx,params.date_peakSWE,params.date_ONS)

In [23]:
prog = pb.ProgressBar(len(params))

In [24]:
while res.ready() == False:
    prog.animate_ipython(res.progress)
    time.sleep(2)

#alert.send_alert('barnhatb@colorado.edu','Processing Nr1 %s has finished'%exname,'Your script has finished')

[****************100%******************]  149921 of 150000 complete                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [25]:
res.done()

True

In [26]:
res.progress

150000

In [27]:
full,melt = zip(*res.result())

## Merge processing results with parameters dataset

In [28]:
idx,trans,evap,sub,ET,PET,meanPET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate = zip(*full)

params = pd.read_pickle('./data/soil_params_multispinup_%s_smex%s_sen.pcl'%(site,method))

# make a date frame of the results
proc = pd.DataFrame()

arrs = [idx,trans,evap,sub,ET,PET,meanPET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,
        sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate]
keys = ['scenidx','trans','evap','snow_sub','ET','PET','meanPET','Q','Qbf','Qrf','rech','sat','p','rz_storage','rz_drainage',
        'unsat_storage','unsat_drainage','sat_def','sat_def_z','sm','gwStore','gwFlux','ETRate']

for arr,key in zip(arrs,keys):
    proc[key] = arr # insert the results

params = pd.merge(params,proc,on='scenidx') # join on file index number
params.dropna(inplace=True) # drop missing values

params.to_hdf('./data/%s_%s_full.hdf'%(site,exname),'df',format='fixed',complevel=5,complib='bzip2',fletcher32=True)

In [29]:
idx,T,E,sub,ET,PET,meanPET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate = zip(*melt)

params = pd.read_pickle('./data/soil_params_multispinup_%s_smex%s_sen.pcl'%(site,method))

# make a date frame of the results
proc = pd.DataFrame()

arrs = [idx,T,E,sub,ET,PET,meanPET,Q,Qbf,Qrf,rech,sat,p,rz_storage,rz_drainage,unsat_storage,unsat_drainage,
        sat_def,sat_def_z,sm,gwStore,gwFlux,ETRate]
keys = ['scenidx','T','E','snow_sub','ET','PET','meanPET','Q','Qbf','Qrf','rech','sat','p','rz_storage','rz_drainage',
        'unsat_storage','unsat_drainage','sat_def','sat_def_z','sm','gwStore','gwFlux','ETRate']

for arr,key in zip(arrs,keys):
    proc[key] = arr # insert the results

params = pd.merge(params,proc,on='scenidx') # join on file index number
params.dropna(inplace=True) # drop missing values

params.to_hdf('./data/%s_%s_melt.hdf'%(site,exname),'df',format='fixed',complevel=5,complib='bzip2',fletcher32=True) # 

In [30]:
alert.send_alert('barnhatb@colorado.edu','Processing P301 %s has finished'%exname,'Your script has finished')